### 📖 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현

- 이 체인은 `영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답`해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").

- 항상 세 개의 이모티콘으로 답장하도록 `FewShotPromptTemplate` 또는 `FewShotChatMessagePromptTemplate`을 사용하여 체인에 예시를 제공하세요.

- 메모리가 작동하는지 확인하려면 체인에 `두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청`하세요.

#### 💡 사용할 라이브러리 및 모듈 import 하기

In [122]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import Runnable, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [123]:
class MyConversationChain:
    def __init__(self, llm, prompt, memory, input_key="question"):
        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(history=self.load_memory)
            | prompt
            | llm
        )

    def load_memory(self, _):
        return self.memory.load_memory_variables({})['history']

    def invoke(self, question):
        answer = self.chain.invoke({self.input_key: question})
        self.memory.save_context({"input": question}, {"output": answer})
        
        return answer

#### 💡 모델(Model) 생성하기

In [124]:
llm = ChatOpenAI(
    temperature=0.1,        # 창의성 (0 ~ 2)
    model='gpt-3.5-turbo',  # 사용 모델 지정 (Default : gpt-3.5-turbo)
)

#### 💡 FewShot 예제 생성하기

In [125]:
# 영화를 이모지를 바꿔달라는 질문의 예시
movie_to_emoji_examples = [
    {
        "question": "영화 탑건을 이모티콘으로 표현해줘!",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "영화 대부를 이모티콘으로 표현해줘!",
        "answer": "👨‍👨‍👦🔫🍝",
    }
]

# 예시 프롬프트 생성
example_template = """
    Human : {question}
    AI : {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

#### 💡 메모리 생성

In [126]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,           # 문자열 기반이 아닌, ChatPromptTemplate 에서 사용할 수 있는 형태로 반환
)

#### 💡 메모리가 사용되도록 하는 Prompt 생성 

In [133]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사람과 대화하기 위해 AI의 도움을 받아야 해."),
    MessagesPlaceholder(variable_name='history'),
    ("human", "{question}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=movie_to_emoji_examples,
    example_prompt=prompt
)

#### 💡 메모리 사용을 위한 함수 정의 

In [128]:
movie_to_emoji_chain = MyConversationChain(llm, few_shot_prompt, memory)

In [129]:
movie_to_emoji_chain.invoke("영화 명량을 이모지로 표현해줘!")

KeyError: 'history'

In [ ]:
movie_to_emoji_chain.invoke("영화 아이언맨을 이모지로 표현해줘!")

In [ ]:
movie_to_emoji_chain.invoke("내가 첫번째로 물어본 영화 이름이 뭐야 ?")